In [36]:
import pandas as pd
import numpy as np

In [37]:
df_original = pd.read_excel('data/vcd_database.xlsx', index_col=1, header=7).iloc[:,1:]
df_original.head(10)

,管理番号,実験日,化合物名,瓶番号,ＣＡＳ\n番号,分子式,測定条件,ファイル名,グラフ
グラフ枚数,,,,,,,,,
1,23,2002-08-02,(R)-(-)-Carvone,NaN,6485-40-1,C10H14O,"CCl4, 0.21 M, 72 um pathlength",rcrv7,rcrv7
2,27,2002-08-05,"(1R,2S,5R)-(-)-Menthol",NaN,2216-51-5,C10H20O,"CCl4, 1.29 M, 72 um pathlength",lmen4,lmen4
3,34,2002-08-07,(-)-Menthyl chloride,NaN,16052-42-9,C10H18Cl,"CCl4, 1.58 M, 72 um pathlength",mmcl3,mmcl3
4,37,2002-08-07,(-)-Borneol,NaN,464-45-9,C10H18O,"CCl4, 0.82 M, 72 um pathlength",mbor3,mbor3
5,40,2002-08-08,(-)-Isoplegol,NaN,89-79-2,C10H16O,"CCl4, 1.46 M, 72 um pathlength",mipg3,mipg3
6,44,2002-08-08,(R)-(+)-Pulegone,NaN,89-82-7,C10H16O,"CCl4, 0.53 M, 72 um pathlength",rplg4,rplg4
7,47,2002-08-09,"(1S,2S,5S)-(-)-Myrtanol",NaN,53369-17-8,C10H18O,"CCl4, 1.19 M, 72 um pathlength",mmyt3,mmyt3
8,64,2002-09-04,(1R)-(-)-Fenchone,NaN,7787-20-4,C10H16O,"CCl4, 0.21 M, 72 um pathlength",rfen2,rfen2
9,69,2002-09-05,(1R)-endo-(+)-Fenchyl alchol,NaN,2217-02-9,C10H18O,"CCl4, 0.83 M, 72 um pathlength",rfea2,rfea2


In [38]:
for x in df_original['化合物名']:
    print(x)
print(len(df_original['化合物名']))

(R)-(-)-Carvone
(1R,2S,5R)-(-)-Menthol
(-)-Menthyl chloride
(-)-Borneol
(-)-Isoplegol
(R)-(+)-Pulegone
(1S,2S,5S)-(-)-Myrtanol
(1R)-(-)-Fenchone
(1R)-endo-(+)-Fenchyl alchol
(1R,2R,3R,5S)-(-)-Isopinocampheol
(1S,2R,5R)-(+)-Isomenthol
(1R)-(-)-Myrtenol
(1R)-(+)-Nopinone
(1S)-(-)-β-Pinene
(1S)-(-)-α-Pinene
(1R)-(-)-Nopol
(1R)-(-)-Myrtenal
(-)-α-Pinene oxide
Myrtenyl chloride
Nopyl chloride
(1R)-(-)-Nopol benzyl ether
(1R,2R,5R)-(+)-2-Hydroxy-3-pinanone
(S)-(+)-2-Pentanol
(S)-(+)-2-Decanol
(S)-(+)-2-Hexanol
(S)-(-)-2-Methyl-1-butanol
(1S)-(-)-α-Pinene
(S)-(+)-5-Methyl-1-heptanol
(S)-(+)-3-Methyl-1-pentanol
(S)-(+)-4-Methyl-1-hexanol
(S)-(-)-β-Citronellol
(1R)-(+)-Camphor
(1R)-(-)-Menthyl acetate
(1S)-(+)-Neomenthyl acetate
(R)-(+)-Citronellal
(S)-(+)-2-Heptanol
(S)-(+)-2-Butanol
Methyl (S)-(-)-lactate
(S)-(+)-2-Nonanol
(R)-(-)-2-Decanol
(R)-(+)-Methyl 3-hydroxy-2-Methylpropionate
Methyl (S)-(+)-mandelate
Ethyl (S)-(+)-mandelate
(S)-(+)-4-Methyl-2-pentanol
(R)-(+)-2-Methyl-1-phenyl-1-propa

In [39]:
odor_names = df_original["化合物名"]
odor_names

グラフ枚数
1                  (R)-(-)-Carvone
2           (1R,2S,5R)-(-)-Menthol
3             (-)-Menthyl chloride
4                      (-)-Borneol
5                    (-)-Isoplegol
                  ...             
159    Cholesterol Ethyl Carbonate
160          Cholesterol n-Caprate
161            Cholesterol Laurate
162          Cholesterol Myristate
163          Cholesterol Palmitate
Name: 化合物名, Length: 163, dtype: object

In [40]:
x = {i: pd.read_excel('data/vcd_database_shadmany.xlsx', sheet_name='Chart%d'%i, index_col=0) 
     for i in range(1,163) if i not in [102]}

### IR and VCD Dataframe ###

In [41]:
ir_values = pd.concat(x).unstack()['ＩＲ']

In [42]:
vcd_values = pd.concat(x).unstack()['ＶＣＤ']

In [43]:
odor_names = pd.DataFrame(odor_names)

In [44]:
vcd_values = odor_names.join(vcd_values, how='inner')

In [45]:
vcd_values

,化合物名,833.18,835.11,837.04,838.97,840.9,842.83,844.76,846.68,848.61,...,1784.0,1785.9,1787.8,1789.8,1791.7,1793.6,1795.5,1797.5,1799.4,1801.3
1,(R)-(-)-Carvone,-4.42160,-0.17114,0.774900,0.11816,0.020507,-0.19042,-0.643550,0.26001,1.25630,...,-1.83590,-2.10660,-2.22820,-2.29900,-2.44180,-2.67500,-2.93970,-3.28240,-3.11540,-2.9106
2,"(1R,2S,5R)-(-)-Menthol",-0.33453,-0.22918,-0.028259,0.11920,0.173950,0.55029,0.689330,0.37365,0.10363,...,-1.37590,-1.50230,-1.52600,-1.50540,-1.53660,-1.55780,-1.61200,-1.70060,-1.64930,-1.6080
3,(-)-Menthyl chloride,0.82022,0.90368,0.987420,1.02980,0.882170,0.86087,0.803950,0.67230,0.77002,...,-0.64529,-0.67880,-0.73834,-0.74649,-0.71688,-0.69571,-0.73602,-0.80505,-0.84014,-0.8572
4,(-)-Borneol,-5.90000,-5.10710,-3.261400,-1.37250,-0.796870,-0.78079,-1.069400,-1.27090,-1.12570,...,1.58720,1.52080,1.50400,1.54320,1.53950,1.53790,1.51700,1.39230,1.41140,1.4685
5,(-)-Isoplegol,-1.09440,-1.05680,-1.009400,-0.81945,-0.732660,-0.40789,0.001892,0.29565,0.48193,...,-1.21000,-1.28170,-1.32530,-1.30010,-1.30100,-1.33530,-1.40210,-1.43750,-1.38760,-1.3881
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
158,β-Cholestanol,9.96820,-3.21090,-4.510200,-0.84326,4.952100,7.27050,2.336400,-1.80660,-0.30615,...,3.54630,3.38910,2.82420,4.39600,4.43550,3.12890,3.07270,2.97850,1.70260,-0.1250
159,Cholesterol Ethyl Carbonate,27.41100,24.21200,16.898000,14.79600,17.809000,17.46400,15.005000,6.01950,-2.51510,...,3.71530,4.84270,4.03120,2.22900,3.43890,4.81490,2.03560,0.24560,2.06980,1.8007
160,Cholesterol n-Caprate,13.99600,14.94000,16.284000,18.93700,18.458000,16.27000,13.448000,8.63110,8.38890,...,-2.31540,-0.84936,0.59643,-0.50268,-1.08930,0.31103,-0.10766,-1.35620,-0.45629,-0.2478
161,Cholesterol Laurate,-4.71330,-18.28900,-25.720000,-24.00000,-13.020000,-14.53500,-22.735000,-22.31200,-19.49100,...,5.10400,4.92190,4.87440,4.86340,4.38580,4.03460,4.43230,4.38550,3.91200,3.3885


In [46]:
def check_for_nans_in_columns(df, columns=None):
    """
    Helper function to check for ``NaN`` in the data frame and raise a ``ValueError`` if there is one.

    :param df: the pandas DataFrame to test for NaNs
    :type df: pandas.DataFrame
    :param columns: a list of columns to test for NaNs. If left empty, all columns of the DataFrame will be tested.
    :type columns: list

    :return: None
    :rtype: None
    :raise: ``ValueError`` of ``NaNs`` are found in the DataFrame.
    """
    if columns is None:
        columns = df.columns

    if pd.isnull(df.loc[:, columns]).any().any():
        if not isinstance(columns, list):
            columns = list(columns)
        raise ValueError(
            "Columns {} of DataFrame must not contain NaN values".format(
                df.loc[:, columns]
                .columns[pd.isnull(df.loc[:, columns]).sum() > 0]
                .tolist()
            )
        )

In [47]:
check_for_nans_in_columns(vcd_values)

ValueError: Columns [833.18, 835.11, 837.04, 838.97, 840.9, 842.83, 844.76, 846.68, 848.61, 850.54, 852.47, 854.4, 856.33, 858.26, 860.18, 862.11, 864.04, 865.97, 867.9, 869.83, 871.76, 873.69, 875.61, 877.54, 879.47, 881.4, 883.33, 885.26, 887.19, 889.11, 891.04, 892.97, 894.9, 896.83, 898.76, 900.69, 902.62, 904.54, 906.47, 908.4, 910.33, 912.26, 914.19, 916.12, 918.04, 919.97, 921.9, 923.83, 925.76, 927.69, 929.62, 931.55, 933.47, 935.4, 937.33, 939.26, 941.19, 943.12, 945.05, 946.97, 948.9, 950.83, 952.76, 954.69, 956.62, 958.55, 960.48, 962.4, 964.33, 966.26, 968.19, 970.12, 972.05, 973.98, 975.9, 977.83, 979.76, 981.69, 983.62, 985.55, 987.48, 989.41, 991.33, 993.26, 995.19, 997.12, 999.05] of DataFrame must not contain NaN values

In [48]:
vcd_values = vcd_values.fillna(0)

In [49]:
check_for_nans_in_columns(vcd_values)

In [50]:
check_for_nans_in_columns(ir_values)

ValueError: Columns [833.18, 835.11, 837.04, 838.97, 840.9, 842.83, 844.76, 846.68, 848.61, 850.54, 852.47, 854.4, 856.33, 858.26, 860.18, 862.11, 864.04, 865.97, 867.9, 869.83, 871.76, 873.69, 875.61, 877.54, 879.47, 881.4, 883.33, 885.26, 887.19, 889.11, 891.04, 892.97, 894.9, 896.83, 898.76, 900.69, 902.62, 904.54, 906.47, 908.4, 910.33, 912.26, 914.19, 916.12, 918.04, 919.97, 921.9, 923.83, 925.76, 927.69, 929.62, 931.55, 933.47, 935.4, 937.33, 939.26, 941.19, 943.12, 945.05, 946.97, 948.9, 950.83, 952.76, 954.69, 956.62, 958.55, 960.48, 962.4, 964.33, 966.26, 968.19, 970.12, 972.05, 973.98, 975.9, 977.83, 979.76, 981.69, 983.62, 985.55, 987.48, 989.41, 991.33, 993.26, 995.19, 997.12, 999.05] of DataFrame must not contain NaN values

In [51]:
ir_values = ir_values.fillna(0)

In [52]:
check_for_nans_in_columns(ir_values)

### Tsfresh data ###

In [53]:
from tsfresh import extract_features

In [54]:
feature_vcdset = extract_features(vcd_values, column_id='化合物名')

Feature Extraction: 100%|██████████| 20/20 [26:46<00:00, 80.31s/it]


In [55]:
feature_vcdset.head()

,1076.2__variance_larger_than_standard_deviation,1076.2__has_duplicate_max,1076.2__has_duplicate_min,1076.2__has_duplicate,1076.2__sum_values,1076.2__abs_energy,1076.2__mean_abs_change,1076.2__mean_change,1076.2__mean_second_derivative_central,1076.2__median,...,1074.2__permutation_entropy__dimension_5__tau_1,1074.2__permutation_entropy__dimension_6__tau_1,1074.2__permutation_entropy__dimension_7__tau_1,1074.2__query_similarity_count__query_None__threshold_0.0,"1074.2__matrix_profile__feature_""min""__threshold_0.98","1074.2__matrix_profile__feature_""max""__threshold_0.98","1074.2__matrix_profile__feature_""mean""__threshold_0.98","1074.2__matrix_profile__feature_""median""__threshold_0.98","1074.2__matrix_profile__feature_""25""__threshold_0.98","1074.2__matrix_profile__feature_""75""__threshold_0.98"
(+)-Chloromethyl isomenthyl ether,0.0,0.0,0.0,0.0,-11.38800,129.686544,NaN,NaN,NaN,-11.38800,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
(+)-Chloromethyl menthyl ether,0.0,0.0,0.0,0.0,3.76570,14.180496,NaN,NaN,NaN,3.76570,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
(+)-Cyclosativene,0.0,0.0,0.0,0.0,-1.88280,3.544936,NaN,NaN,NaN,-1.88280,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
(+)-Isoplegol,0.0,0.0,0.0,0.0,2.13870,4.574038,NaN,NaN,NaN,2.13870,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
(+)-Longifolene,0.0,0.0,0.0,0.0,1.52952,1.256202,0.4159,0.4159,NaN,0.76476,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [56]:
feature_vcdset = feature_vcdset.replace([np.inf, -np.inf], np.nan)

In [57]:
feature_vcdset = feature_vcdset.fillna(0)

In [58]:
feature_vcdset.head()

,1076.2__variance_larger_than_standard_deviation,1076.2__has_duplicate_max,1076.2__has_duplicate_min,1076.2__has_duplicate,1076.2__sum_values,1076.2__abs_energy,1076.2__mean_abs_change,1076.2__mean_change,1076.2__mean_second_derivative_central,1076.2__median,...,1074.2__permutation_entropy__dimension_5__tau_1,1074.2__permutation_entropy__dimension_6__tau_1,1074.2__permutation_entropy__dimension_7__tau_1,1074.2__query_similarity_count__query_None__threshold_0.0,"1074.2__matrix_profile__feature_""min""__threshold_0.98","1074.2__matrix_profile__feature_""max""__threshold_0.98","1074.2__matrix_profile__feature_""mean""__threshold_0.98","1074.2__matrix_profile__feature_""median""__threshold_0.98","1074.2__matrix_profile__feature_""25""__threshold_0.98","1074.2__matrix_profile__feature_""75""__threshold_0.98"
(+)-Chloromethyl isomenthyl ether,0.0,0.0,0.0,0.0,-11.38800,129.686544,0.0000,0.0000,0.0,-11.38800,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
(+)-Chloromethyl menthyl ether,0.0,0.0,0.0,0.0,3.76570,14.180496,0.0000,0.0000,0.0,3.76570,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
(+)-Cyclosativene,0.0,0.0,0.0,0.0,-1.88280,3.544936,0.0000,0.0000,0.0,-1.88280,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
(+)-Isoplegol,0.0,0.0,0.0,0.0,2.13870,4.574038,0.0000,0.0000,0.0,2.13870,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
(+)-Longifolene,0.0,0.0,0.0,0.0,1.52952,1.256202,0.4159,0.4159,0.0,0.76476,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [59]:
#Dropping the columns with only zeros
feature_vcdset = feature_vcdset.loc[:, (feature_vcdset**2).sum() != 0]

In [60]:
feature_vcdset.head()

,1076.2__variance_larger_than_standard_deviation,1076.2__sum_values,1076.2__abs_energy,1076.2__mean_abs_change,1076.2__mean_change,1076.2__mean_second_derivative_central,1076.2__median,1076.2__mean,1076.2__length,1076.2__standard_deviation,...,1074.2__lempel_ziv_complexity__bins_3,1074.2__lempel_ziv_complexity__bins_5,1074.2__lempel_ziv_complexity__bins_10,1074.2__lempel_ziv_complexity__bins_100,1074.2__fourier_entropy__bins_2,1074.2__fourier_entropy__bins_3,1074.2__fourier_entropy__bins_5,1074.2__fourier_entropy__bins_10,1074.2__fourier_entropy__bins_100,1074.2__permutation_entropy__dimension_3__tau_1
(+)-Chloromethyl isomenthyl ether,0.0,-11.38800,129.686544,0.0000,0.0000,0.0,-11.38800,-11.38800,1.0,0.00000,...,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
(+)-Chloromethyl menthyl ether,0.0,3.76570,14.180496,0.0000,0.0000,0.0,3.76570,3.76570,1.0,0.00000,...,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
(+)-Cyclosativene,0.0,-1.88280,3.544936,0.0000,0.0000,0.0,-1.88280,-1.88280,1.0,0.00000,...,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
(+)-Isoplegol,0.0,2.13870,4.574038,0.0000,0.0000,0.0,2.13870,2.13870,1.0,0.00000,...,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
(+)-Longifolene,0.0,1.52952,1.256202,0.4159,0.4159,0.0,0.76476,0.76476,2.0,0.20795,...,1.0,1.0,1.0,1.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0


In [61]:
#tells us how many 0's there are and sorts the columns by index
#(feature_vcdset==0).sum().value_counts().sort_index()

In [62]:
#check how many columns have zero varience
((feature_vcdset.var() <= 0) == True).sum()

995

In [63]:
feature_vcdset = feature_vcdset.loc[:, ((feature_vcdset.var() <= 0) == False)]

In [64]:
feature_vcdset.head()

,1076.2__variance_larger_than_standard_deviation,1076.2__sum_values,1076.2__abs_energy,1076.2__mean_abs_change,1076.2__mean_change,1076.2__mean_second_derivative_central,1076.2__median,1076.2__mean,1076.2__length,1076.2__standard_deviation,...,1074.2__lempel_ziv_complexity__bins_2,1074.2__lempel_ziv_complexity__bins_3,1074.2__lempel_ziv_complexity__bins_5,1074.2__lempel_ziv_complexity__bins_10,1074.2__fourier_entropy__bins_2,1074.2__fourier_entropy__bins_3,1074.2__fourier_entropy__bins_5,1074.2__fourier_entropy__bins_10,1074.2__fourier_entropy__bins_100,1074.2__permutation_entropy__dimension_3__tau_1
(+)-Chloromethyl isomenthyl ether,0.0,-11.38800,129.686544,0.0000,0.0000,0.0,-11.38800,-11.38800,1.0,0.00000,...,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
(+)-Chloromethyl menthyl ether,0.0,3.76570,14.180496,0.0000,0.0000,0.0,3.76570,3.76570,1.0,0.00000,...,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
(+)-Cyclosativene,0.0,-1.88280,3.544936,0.0000,0.0000,0.0,-1.88280,-1.88280,1.0,0.00000,...,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
(+)-Isoplegol,0.0,2.13870,4.574038,0.0000,0.0000,0.0,2.13870,2.13870,1.0,0.00000,...,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
(+)-Longifolene,0.0,1.52952,1.256202,0.4159,0.4159,0.0,0.76476,0.76476,2.0,0.20795,...,1.0,1.0,1.0,1.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0


In [65]:
#((feature_vcdset.var() >= 1) == True).sum()
print(feature_vcdset.var().max())
print(feature_vcdset.var().min())

9616152258765.818
7.275585092306096e-15


### PCA Data ###

In [66]:
from sklearn.decomposition import PCA

In [67]:
pca = PCA(n_components = 15)

In [77]:
#vcd_values = vcd_values.iloc[:, 1:]
pca_vcd_values = pd.DataFrame(index=vcd_values.index, columns=range(15))
pca_vcd_values[:] = pca.fit_transform(vcd_values)

C:\Users\Owner\Anaconda3\lib\site-packages\sklearn\utils\validation.py:1679: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['float']. An error will be raised in 1.2.
  FutureWarning,


In [78]:
pca_vcd_values

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
1,7.464317,-43.591126,-8.046428,-0.574730,-3.580253,5.872906,-2.689293,0.910200,3.263804,1.241826,-1.818156,-10.064771,-5.520251,6.959874,0.015222
2,-15.984788,-48.409755,11.599679,-6.459996,-2.895888,-21.817191,7.987372,30.168362,-4.855941,-27.966820,-15.593914,18.881434,6.681510,-3.868337,-18.297382
3,-5.850617,-48.612997,25.383460,-1.972610,-9.623229,-12.079313,-5.014843,9.790831,-11.125835,-10.948899,12.772647,-4.067735,4.195325,0.328834,0.769539
4,0.756580,-25.884643,-4.987043,6.148153,-18.167910,7.294057,-16.954401,9.166869,10.159980,8.512613,22.090471,-60.186692,5.242115,9.027852,-17.818851
5,-12.599590,-33.717238,-12.966523,-18.479083,5.945712,-6.208224,-4.337389,10.783434,-12.640215,1.773678,1.927767,6.934410,5.856864,8.256831,-9.983370
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
158,6.415996,-18.840982,20.303654,-29.238431,-26.058053,8.328638,-8.520263,19.753265,24.650637,37.600200,-1.058150,-16.202383,37.119465,-1.958009,-10.760869
159,230.771312,139.215929,-45.562654,-28.445263,7.019739,-30.857064,18.819414,-6.542079,7.706868,-5.896486,7.641171,-5.184812,5.986100,-34.832364,-7.767732
160,181.677425,33.151223,72.048835,-49.147260,19.664894,20.812248,-3.977718,5.351023,14.182108,-2.607744,6.333383,-0.250627,-7.471805,19.548504,-8.367786
161,-130.152846,164.035229,61.091725,-37.136634,-13.847156,14.540341,-4.944996,-3.105042,2.508837,1.700014,5.700880,8.617425,-4.495039,22.148967,2.382606


In [74]:
pca.fit(vcd_values)

C:\Users\Owner\Anaconda3\lib\site-packages\sklearn\utils\validation.py:1679: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['float']. An error will be raised in 1.2.
  FutureWarning,


PCA(n_components=15)

In [75]:
pca.explained_variance_ratio_.cumsum()

array([0.41520387, 0.59295937, 0.6666533 , 0.71686009, 0.74416619,
       0.76945484, 0.79056538, 0.80899373, 0.82595596, 0.84040002,
       0.8537301 , 0.86544691, 0.87570667, 0.88518658, 0.89369171])

### NewSection Come back to

In [ ]:
from tsfresh import extract_relevant_features

In [ ]:
feature_vcdset_rel = extract_relevant_features(vcd_values, )

In [ ]:
feature_vcdset

In [ ]:
ir_values = odor_names.join(ir_values, how='inner')

In [ ]:
ir_values

In [ ]:
check_for_nans_in_columns(ir_values)

In [ ]:
ir_values = ir_values.fillna(0)

In [ ]:
check_for_nans_in_columns(ir_values)

In [ ]:
features_irset = extract_features(ir_values, column_id='化合物名')

In [ ]:
features_irset

In [ ]:
features_irset.to_csv("IR_features")

In [ ]:
import xlrd

In [ ]:
x = xlrd.open_workbook('data/vcd_database_shadmany.xlsx')

In [ ]:
#list(x.keys())

In [ ]:
#vcd = pd.DataFrame(index=smiles, columns=wavelengths)
#ir = pd.DataFrame(index=smiles, columns=wavelengths)

In [ ]:
#CIDs = pd.read_csv('data/vcd_data_shadmany.csv')